In [33]:
import glob
import random_cropping
import pandas as pd
from PIL import Image
import random
from math import floor, ceil

In [62]:
desired_crop_size = 1024

DATA_PATH = "C:/Users/clieshou/Documents/Sogeti/The Ocean Cleanup/Plasticdebris_data/"
image_paths = glob.glob(DATA_PATH + 'images/shipcam/*')
OUTPUT_PATH = "shipcam_emptycrops/images"
label_file_name = "labels_split.csv"

In [63]:
df = pd.read_csv(os.path.join(DATA_PATH, label_file_name), delimiter=';')
df['filename'] = df['filename'].str.lower()

In [64]:
def define_center_sampling_region(image_width, image_height,desired_crop_size):
    """
    Steps:
    Determines from what region the center of the crop can be randomly selected.
    It does this by compensating desired crop size for the size of the bounding box, to avoid it not being fully included.
    It compensates for the fact that bounding boxes may be close to the edge of the image, didn't get to it yet
    """

    #we want to select the center of the crop randomly, but have to make sure that the entire bb falls within the crop
    center_sampling_region_xmin = desired_crop_size/2 
    center_sampling_region_ymin = desired_crop_size/2 
    center_sampling_region_xmax = image_width - desired_crop_size/2 
    center_sampling_region_ymax = image_height - desired_crop_size/2 

    #randomly getting the center within the allowed frame, using seed to get the same each time for the same bounding box. Not sure if there are any reasons not to do this
    # random.seed()
    crop_xcenter = random.randint(floor(center_sampling_region_xmin), ceil(center_sampling_region_xmax))
    # random.seed(bb_ymin)
    crop_ycenter = random.randint(floor(center_sampling_region_ymin), ceil(center_sampling_region_ymax))

    #make sure the center fall within the acceptable frame, where the entire crop will fall within the image
    if crop_xcenter < desired_crop_size/2:
        crop_xcenter = desired_crop_size/2
    if crop_xcenter > (image_width - desired_crop_size/2):
        crop_xcenter = (image_width - desired_crop_size/2)
    if crop_ycenter < desired_crop_size/2:
        crop_ycenter = desired_crop_size/2
    if crop_ycenter > (image_height - desired_crop_size/2):
        crop_ycenter = (image_height - desired_crop_size/2)

    return crop_xcenter, crop_ycenter

In [65]:
def generate_crop_per_bb(image_path, df_image, desired_crop_size):
    """
    This functions aim is to obtain crops of a specified size from images of any size. 
    It does this by first checking whether the bounding box of interest is larger or smaller than desired.
    If larger, it downsamples the entire image first.
    Then, we determine from what region the center of the crop can be sampled such that the bb will always fully fall within the image.

    input:
    - i: index of the object of interest for the current image
    - image: the path to the image
    - df_image: subset of the total df, only contains rows of this image
    - downsample_marging: how much smaller should the bounding box be than the image, 1 is the same, 2 is half
    
    returns:
    - crop: cropped image
    - crop_coordinates: xmin, ymin, xmax and ymax of the crop within the original image
    """

    image_to_crop = Image.open(image_path)
    #initialize variables for the required data
    # bb_data = df_image.iloc[i, :]
    # bb_xmin, bb_ymin, bb_xmax, bb_ymax = bb_data['xmin'], bb_data['ymin'], bb_data['xmax'], bb_data['ymax']
    
    #check if a bounding box is too large for the desired crop size, and if so, sample it down
    
    #determine what the center of the crop should be
    image_width, image_height = image_to_crop.size
    center_sampling_region_xcenter, center_sampling_region_ycenter = define_center_sampling_region(image_width, image_height, desired_crop_size)

    #determine crop coordinates, then crop
    crop_xmin = center_sampling_region_xcenter - desired_crop_size/2
    crop_ymin = center_sampling_region_ycenter - desired_crop_size/2
    crop_xmax = center_sampling_region_xcenter + desired_crop_size/2
    crop_ymax = center_sampling_region_ycenter + desired_crop_size/2

    crop_coordinates = (crop_xmin, crop_ymin, crop_xmax, crop_ymax)
    crop = image_to_crop.crop(crop_coordinates)

    return crop

In [70]:
desired_crop_size = 1024
df_image = df[df['filename'] == image_path.split("/")[-1]]

In [73]:
crops_created = 0
while crops_created < 1000:
    
    random_select = random.randint(0, len(image_paths))
    image_path = image_paths[random_select]
    image_name = image_path.split("/")[-1].split("\\")[1]

    crop = generate_crop_per_bb(image_path, df_image, desired_crop_size)

    df_image = df[df['filename'] == image_name]

    image_path_crop = os.path.join(OUTPUT_PATH, image_name.split('.')[0] + '_crop.jpg')

    print(image_path_crop)
    crop.save(image_path_crop)

    crops_created += 1

shipcam_emptycrops/images\dsc00968_crop.jpg
shipcam_emptycrops/images\dsc00968_crop.jpg
shipcam_emptycrops/images\dsc00968_crop.jpg
shipcam_emptycrops/images\dsc00968_crop.jpg
shipcam_emptycrops/images\dsc00968_crop.jpg
shipcam_emptycrops/images\dsc00968_crop.jpg
shipcam_emptycrops/images\dsc00968_crop.jpg
shipcam_emptycrops/images\dsc00968_crop.jpg
shipcam_emptycrops/images\dsc00968_crop.jpg
shipcam_emptycrops/images\dsc00968_crop.jpg
shipcam_emptycrops/images\dsc00968_crop.jpg
shipcam_emptycrops/images\dsc00968_crop.jpg


KeyboardInterrupt: 

In [ ]:
crop

In [11]:
image_path = image_paths[0]